In [1]:
import os
import shutil
import great_expectations as gx
import great_expectations.expectations as gxe
import sys
import os

# 动态获取当前 Notebook 所在的目录
# 如果是在 papermill/airflow 中运行，getcwd() 通常能拿到正确路径
current_dir = os.getcwd() 

if current_dir not in sys.path:
    sys.path.append(current_dir)

# --- 修改 1: 适配本地路径 ---
# 使用当前目录下的 gx_configs 文件夹
gx_root_dir = os.path.abspath("./gx_configs") 

if os.path.exists(gx_root_dir):
    print(f"强制清空本地目录: {gx_root_dir}")
    # 使用标准 python 库删除目录
    shutil.rmtree(gx_root_dir) 

# 重新创建
os.makedirs(gx_root_dir, exist_ok=True)

def get_context():
    return gx.get_context(context_root_dir=gx_root_dir)

context = get_context()
print(gx_root_dir)

强制清空本地目录: /Users/luhui/Documents/github/09_reddit_posts_ELT/pyspark/gx_configs
/Users/luhui/Documents/github/09_reddit_posts_ELT/pyspark/gx_configs


/opt/anaconda3/lib/python3.13/site-packages/great_expectations/data_context/store/_store_backend.py:88: DeprecationWarning: 'parseString' deprecated - use 'parse_string'
  parsed_store_backend_id = store_backend_id_file_parser.parseString(


In [2]:
tables_to_validate = [
    {"table": "reddit_posts_bz", "suite": "reddit_posts_bz_suite"}
]

for item in tables_to_validate:
    table_name = item["table"]
    suite_name = item["suite"]
    
    # 1.10+ 获取或创建 Suite 的标准写法
    try:
        # 使用 .suites.get 获取
        suite = context.suites.get(name=suite_name)
        print(f"✅ 已加载现有 Suite: {suite_name}")
    except Exception:
        # 使用 .suites.add 和 gx.ExpectationSuite 创建
        # 这会自动在 Files/gx_config/expectations/ 下生成对应的 .json 文件
        suite = context.suites.add(gx.ExpectationSuite(name=suite_name))
        print(f"✨ 已成功新建并持久化 Suite: {suite_name}")

print("\n所有 Suite 初始化完成。")

✨ 已成功新建并持久化 Suite: reddit_posts_bz_suite

所有 Suite 初始化完成。


In [3]:
table_rules_mapping = {
    "reddit_posts_bz": [
        gxe.ExpectTableColumnsToMatchSet(
            column_set=[
                "post_id",
                "title",
                "author",
                "score",
                "upvote_ratio",
                "comments",
                "flair",
                "is_video",
                "is_self",
                "domain",
                "url",
                "created_utc",
                "selftext",
                "extracted_time",
                "load_time",
                "_dq_batch_id"
            ],
            exact_match=True 
            ),
        gxe.ExpectColumnValuesToNotBeNull(column="post_id"),
        gxe.ExpectColumnValuesToNotBeNull(column="title"),
        gxe.ExpectColumnValuesToMatchStrftimeFormat(
            column="created_utc",
            # 匹配 "2026-01-07 06:01:42"
            strftime_format="%Y-%m-%d %H:%M:%S"
        ),
        gxe.ExpectColumnValuesToBeInSet(
            column="is_video",
            value_set=["True", "False"]
        ),
        gxe.ExpectColumnValuesToBeInSet(
            column="is_self",
            value_set=["True", "False"]
        ),
        gxe.ExpectColumnValuesToBeBetween(
            column="upvote_ratio",
            min_value=0.0,
            max_value=1.0
        )

    ]    
}

In [4]:
def initialize_all_suites(context, rules_mapping):
    for table_name, expectations in rules_mapping.items():
        suite_name = f"{table_name}_suite"
        
        # 1. 获取或创建 Suite
        try:
            suite = context.suites.get(name=suite_name)
            suite.expectations = [] 
            print(f"🔄 更新现有 Suite: {suite_name}")
        except Exception:
            suite = context.suites.add(gx.ExpectationSuite(name=suite_name))
            print(f"✨ 创建新 Suite: {suite_name}")

        # 2. 为该表添加定义的每一条规则
        for exp in expectations:
            # add_expectation 在 1.x 中会自动去重（如果规则完全一样）
            suite.add_expectation(exp)
        
        print(f"   已添加 {len(expectations)} 条规则到 {suite_name}")

In [5]:
initialize_all_suites(context, table_rules_mapping)

🔄 更新现有 Suite: reddit_posts_bz_suite
   已添加 7 条规则到 reddit_posts_bz_suite
